### iNaturlist 
* [#179](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/179) "Koppla SAT till artobservationer"
* [this Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/SAT179_iNaturalist.ipynb)

In [2]:
from shapely.geometry import shape, Polygon, MultiPolygon

def fetch_inat(polygon_geom, iconic_taxa=None, d1=None, d2=None, per_page=200, max_pages=5):
    geom = shape(polygon_geom)

    # Om det är multipolygon, ta unionen och välj största polygonen
    if isinstance(geom, MultiPolygon):
        geom = max(geom.geoms, key=lambda a: a.area)

    # Om det är LineString, buffra lite
    if geom.geom_type == "LineString":
        geom = geom.buffer(0.001)  # ca 100 m, justera

    if geom.geom_type != "Polygon":
        raise ValueError(f"Kan inte hantera geometri av typ {geom.geom_type}")

    # Plocka ut yttre ringen
    ring = list(geom.exterior.coords)
    polygon_param = ",".join([f"{lng},{lat}" for (lng, lat) in ring])

    params = {
        "polygon": polygon_param,
        "verifiable": "true",
        "order_by": "observed_on",
        "per_page": per_page
    }
    if iconic_taxa:
        params["iconic_taxa"] = iconic_taxa
    if d1: params["d1"] = d1
    if d2: params["d2"] = d2

    results = []
    for page in range(1, max_pages + 1):
        params["page"] = page
        r = requests.get("https://api.inaturalist.org/v1/observations", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        obs = data.get("results", [])
        results.extend(obs)
        if len(obs) < per_page:
            break
        time.sleep(0.2)
    return results


In [4]:
import time, requests
from shapely.geometry import shape, Polygon, MultiPolygon, LineString
from shapely.ops import unary_union

INAT_URL = "https://api.inaturalist.org/v1/observations"

def _inat_query_for_polygon(poly: Polygon, params_base: dict, per_page=200, max_pages=5, pause=0.2):
    """Kör iNaturalist-fråga för EN polygon (ytre ring) och returnerar en lista med obs."""
    # Förenkla svagt för att minska antalet koordinater i URL:en (justera vid behov)
    poly = poly.simplify(0.0001, preserve_topology=True)
    ring = list(poly.exterior.coords)  # Shapely exterior är redan stängd (första==sista)
    polygon_param = ",".join(f"{lng},{lat}" for (lng, lat) in ring)

    params = dict(params_base)
    params.update({
        "polygon": polygon_param,
        "verifiable": "true",
        "order_by": "observed_on",
        "per_page": per_page
    })

    results = []
    for page in range(1, max_pages + 1):
        params["page"] = page
        r = requests.get(INAT_URL, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        obs = data.get("results", [])
        results.extend(obs)
        if len(obs) < per_page:
            break
        time.sleep(pause)  # snäll mot API:t
    return results

def fetch_inat_all_parts(geom_like,
                         iconic_taxa=None, d1=None, d2=None,
                         per_page=200, max_pages=5, pause=0.2):
    """
    Tar emot vilken geometri som helst (LineString/Polygon/MultiPolygon),
    buffrar vid behov, loopar över alla delpolygoner och returnerar deduplade obs.
    """
    g = shape(geom_like)

    # Om det är en linje, buffra (justera buffert efter behov)
    if isinstance(g, LineString):
        g = g.buffer(0.001)  # ~100 m

    # Om blandade/geometrisamlingar → slå ihop
    if g.geom_type not in ("Polygon", "MultiPolygon"):
        g = unary_union(g)

    # Samla alla polygoner
    if isinstance(g, Polygon):
        polys = [g]
    elif isinstance(g, MultiPolygon):
        # valfritt: filtrera bort super-små öar som mest ger brus
        polys = [p for p in g.geoms if p.area > 1e-10]
    else:
        raise ValueError(f"Kan inte hantera geometri av typ {g.geom_type}")

    params_base = {}
    if iconic_taxa: params_base["iconic_taxa"] = iconic_taxa
    if d1: params_base["d1"] = d1
    if d2: params_base["d2"] = d2
    # Exempel: params_base["quality_grade"] = "research"

    # Hämta för varje delpolygon och dedupla på observation-id
    all_obs = {}
    for poly in polys:
        try:
            part_obs = _inat_query_for_polygon(
                poly, params_base, per_page=per_page, max_pages=max_pages, pause=pause
            )
            for o in part_obs:
                all_obs[o["id"]] = o
        except requests.HTTPError as e:
            # Vanligt fel: URL blir för lång pga för många koordinater → höj simplify()
            print(f"[iNat] Fel för en delpolygon: {e}")

    return list(all_obs.values())


In [5]:
# Exempel-filter
d1, d2 = "2024-03-01", "2024-10-31"

plants  = fetch_inat_all_parts(search_poly.__geo_interface__, iconic_taxa="Plantae", d1=d1, d2=d2)
animals = []
for tx in ["Aves","Mammalia","Reptilia","Amphibia","Insecta","Arachnida","Mollusca","Animalia"]:
    animals += fetch_inat_all_parts(search_poly.__geo_interface__, iconic_taxa=tx, d1=d1, d2=d2, max_pages=3)
# Dedupla djur ifall en taxa överlappar:
animals = {o["id"]: o for o in animals}.values()


In [6]:
import requests, folium
from folium.plugins import TimestampedGeoJson

# --- 1) Hämta 20 senaste observationerna från iNaturalist nära Möja (exempel) ---
params = {
    "lat": 59.408,
    "lng": 18.79,
    "radius": 10,          # km
    "per_page": 20,
    "order_by": "created_at",
    "order": "desc"
}
data = requests.get("https://api.inaturalist.org/v1/observations", params=params).json()

# --- 2) Bygg GeoJSON features ---
features = []
for o in data["results"]:
    if not o.get("geojson"): 
        continue
    lat, lon = o["geojson"]["coordinates"][1], o["geojson"]["coordinates"][0]
    date = o.get("observed_on") or o.get("created_at")
    taxon = o.get("taxon") or {}
    cname = taxon.get("preferred_common_name") or taxon.get("name") or "okänd"
    url = f"https://www.inaturalist.org/observations/{o['id']}"
    popup = f"<b>{cname}</b><br>{date}<br><a href='{url}' target='_blank'>Visa</a>"
    features.append({
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [lon, lat]},
        "properties": {
            "time": date,
            "popup": popup,
            "icon": "circle",
            "iconstyle": {"color": "green", "fillColor": "green", "fillOpacity": 0.6}
        }
    })

geojson = {
    "type": "FeatureCollection",
    "features": features
}

# --- 3) Folium-karta + animerad tidslider ---
m = folium.Map(location=[59.408, 18.79], zoom_start=11)
TimestampedGeoJson(
    geojson,
    period="P1D",           # 1 dag per steg
    duration="P1D",         # varje punkt visas 1 dag
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options="YYYY-MM-DD",
    time_slider_drag_update=True
).add_to(m)

m.save("inat_realtime_poc.html")


SyntaxError: invalid syntax (1419883496.py, line 1)